In [1]:
import pickle
from random import uniform
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from tqdm.auto import tqdm
from fake_useragent import UserAgent

In [2]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

ua = UserAgent()
driver = webdriver.Chrome(options=options)
# driver.implicitly_wait(0)

In [3]:
with open("actor_dict.pickle", "rb") as file:
    actor_dict = pickle.load(file)

In [12]:
for actor_id, info_dict in tqdm(actor_dict.items()):
    if info_dict.get('全名') is not None or "search" in info_dict['page_link']:
        continue

    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": ua.random})
    driver.get(info_dict['page_link'])
    start_time = time()
    
    try:
        # fetch all info in plain text
        actor_info = driver.find_element_by_class_name("info")
        info_dict.update(dict([line.split(": ", 1) for line in actor_info.text.split("\n")]))
        info_dict['全名'] = driver.find_element_by_xpath("//div[@id=\"content\"]/h1").text
        info_dict['影人简介'] = driver.find_element_by_xpath("//div[@id='intro']/div[@class='bd']").text
        info_dict['photo'] = driver.find_element_by_xpath("//div[@id=\"headline\"]/div[@class='pic']/a/img").get_attribute("src")
    except NoSuchElementException:
        print(driver.current_url)
        break
        
    driver.delete_all_cookies()
    duration = time() - start_time
    if duration < 10.0:
        sleep(10.0 - duration)

https://sec.douban.com/b?r=https%3A%2F%2Fmovie.douban.com%2Fcelebrity%2F1151972%2F



In [ ]:
with open("actor_info_{}-{}.pickle", "wb") as file:
    pickle.dump(actor_dict[start_idx:end_idx], file)

In [6]:
driver.close()